In [2]:
import pandas as pd
import xlwings as xw
from win32com.client import Dispatch 

In [1]:
import os 

os.mkdir(r'C:\Users\yefan\WPSDrive\339514258\WPS云盘\共享文件夹 \东方生物2024年年审\2、试算\2024年试算\test')

In [1]:
import os

def is_file_open(file_path):
    try:
        with open(file_path, 'r+'):
            pass
    except (IOError, PermissionError):
        return True
    return False

# 示例
file_path = r"D:\audit_project\AUTO_TB\映射模板设计.xlsx"
if is_file_open(file_path):
    print(f"文件 {file_path} 已被打开。")
else:
    print(f"文件 {file_path} 未被打开。")


文件 D:\audit_project\AUTO_TB\映射模板设计.xlsx 已被打开。


In [17]:
def VBA_update_data(path,sheet_name,update_details,engine=None):

    path=path 
    data=update_details[['单元格','金额']].values.tolist()
    print(data)

    #VBA代码
    code_vba='''
    Sub BatchUpdate(updates As Variant, sheetName As String)
    Dim i As Long
    Dim targetSheet As Worksheet
    Dim cellAddress As String
    Dim cellValue As Variant

    ' 获取目标工作表
    On Error Resume Next
    Set targetSheet = ThisWorkbook.Sheets(sheetName)
    On Error GoTo 0

    ' 遍历数组，更新单元格
    For i = LBound(updates, 1) To UBound(updates, 1)
        cellAddress = updates(i,0)
        cellValue = updates(i,1)
        targetSheet.Range(cellAddress).Value = cellValue
    Next i
    End Sub

    '''
    if engine is None:     # xlwings 打开 wps 
        xl = xw._xlwindows.COMRetryObjectWrapper(Dispatch("Ket.Application")) 
        impl = xw._xlwindows.App(visible=False, add_book=False, xl=xl)     
        with xw.App(visible=True, add_book=False, impl=impl) as app:
            wb=app.books.open(path)
            vb_object = wb.api.VBProject.VBComponents.Add(1)  # 1 表示标准模块
            vb_object.CodeModule.AddFromString(code_vba)
            wb.macro("BatchUpdate")(data,sheet_name)
            wb.api.VBProject.VBComponents.Remove(vb_object)
            wb.save()
            wb.close()
    else:     # xlwings 打开 excel
        with xw.App(visible=True) as app:
            wb=app.books.open(path)
            vb_object = wb.api.VBProject.VBComponents.Add(1)  # 1 表示标准模块
            vb_object.CodeModule.AddFromString(code_vba)
            wb.macro("BatchUpdate")(data,sheet_name)
            wb.api.VBProject.VBComponents.Remove(vb_object)
            wb.save()
            wb.close()

df=pd.DataFrame([['A1',0],['B3',10086]])
df.columns=['单元格','金额']
VBA_update_data(path=r"D:\audit_project\AUTO_TB\test.xlsx",sheet_name='test',update_details=df,engine='excel')

[['A1', 0], ['B3', 10086]]
